# 代码解析

## 整体思路：
1. 构建可能的会话情景，并在相应的情景填充可能的回复语句。（整体为字典形式，会话情景为索引，相对应的回复语句集合构成这个会话情景所对应的列表）
2. 对输入的第一句话，通过上面所构建的“图鉴”，看是否存在与之对应的情景，并从中提取出特定的谈论对象。
3. 将上一步提取出的特定对象填充入回复语句中，并通过随机选取，输出回复。

### 对单一情景，判断输入是否与情景匹配并得到情景占位符所对应的特定对象

In [1]:
def is_variable(pat):    #判断输入是否为特定形式
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])#and前判断输入是否为特定标识开头；and后判断后续内容是否书写规范

In [2]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]     #第一个if判断模式元素是否为占位符
    else:
        if pattern[0] != saying[0]: return False     #第二个if判断模式与输入是否匹配
        else:
            return pat_match(pattern[1:], saying[1:])    #这里有一个逻辑缺陷，就是由于第一个判断，倘若模式中第一个元素就是占位符，那么
        #程序将默认输入与模式是匹配的，而没有对后续的内容进行检验。在这里递归的停止依据是找到找到占位符，或模式输入不匹配。若输入
        #与模式足够长，又互相匹配且不存在占位符，那么递归将会永远进行下去

In [3]:
pat_match("?X equal ?X".split(), "2+2 equals 8+2".split())#在这里可以明显看出，输入与模板其实并不匹配，只是由于模板的第一个元素满足了
#程序中第一个if的判断。

('?X', '2+2')

In [5]:
pat_match("?X equals ?Y".split(), "2+2 equals 8+2".split())#这里也显示出此时的程序并不适用于多个占位符

('?X', '2+2')

### 程序改进如下

In [6]:
def pat_match(pattern, saying):
    if not pattern or not saying: return [] #对于空列表不考虑，也是递归停止的判断依据
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])#有了这一步，递归就不会因找到了一个占位符而停止
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [7]:
pat_match("?X equal ?Y".split(), "2+2 equals 8+2".split())#递归在元素equal处停止

[('?X', '2+2')]

In [8]:
pat_match("?X equals ?Y".split(), "2+2 equals 8+2".split())

[('?X', '2+2'), ('?Y', '8+2')]

### 将占位符与特定对象的对应关系转化为字典形式

In [9]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}#patterns占位符与特定对象的对应，k为占位符，v特定对象！！！！！！！！！

In [10]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)#get语句中第一个是返回索引rule[0]对应元素，第二个指若没有
#rule[0]索引，则返回原值！！！！！！！

In [11]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

### 对于占位符的进一步改进

In [12]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])#此处的处理和上面的is_variable如出一辙，这里的目的也只是对碰到
#占位符的情况进行进一步的细分

In [13]:
from collections import defaultdict  #增加模组

In [14]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])    #在有标记符的情况下判断是否满足第一种情况
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)     #在有标记符出现的情况下判断是否满足第二种情况
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])#match为得到的占位符元素对应关系，index为saying中的关键索引
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [15]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]#对pattern的拆分
    seg_pat = seg_pat.replace('?*', '?')#替换操作，由于上面已经证明了pattern中必然存在占位符'?'，所以必然可以顺利执行

    if not rest: return (seg_pat, saying), len(saying)  #如果是仅有占位符的特殊情况，直接返回saying
    
    for i, token in enumerate(saying):#i是索引值，token是索引所对应的元素！！！！！！！！！！！！！
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i#返回对应关系和关键的索引值
    
    return (seg_pat, saying), len(saying)#模式不匹配的情况

def is_match(rest, saying):#检测找到占位符所对应的部分之后，剩下的部分是否与模式匹配
    if not rest and not saying:#递归终止条件
        return True
    if not all(a.isalpha() for a in rest[0]):#元素是否书写规范
        return True
    if rest[0] != saying[0]:#模式不匹配
        return False
    return is_match(rest[1:], saying[1:])

In [16]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}#patterns为def pat_match_with_seg的输出

In [17]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [18]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [19]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [20]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [21]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [22]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

## 问题一

In [2]:
import random

In [3]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [4]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [5]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [6]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [7]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [8]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)  
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying) 


In [9]:
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])


In [10]:
def response(saying,rules_list):
    for key in rules_list.keys():#字典索引的列表！！！！！！！看rules_list里是否有相应的情景模式
        get_pattern = pat_match_with_seg(key.split(),saying.split())
        
        if get_pattern :
            if len(get_pattern[0][1])!=len(saying.split()):###避免这种情况：[?*x,saying.split()]这种情况将会导致无法选择合适的情景
                select = random.choice(rules_list[key])
                decision = subsitite(select.split(), pat_to_dict(get_pattern))
                return ' '.join(decision)
        else: print("sounds intresting")
            
   

In [13]:
rules_list = { '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']}
response("sometime I feel sad",rules_list)

'Do you often feel sad ?'

## 问题二

In [14]:
import random

In [15]:
import jieba

ModuleNotFoundError: No module named 'jieba'

In [16]:
from collections import defaultdict

In [17]:
fail = [True, None]

def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])


In [18]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [19]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])


In [20]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [21]:
def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    pat = pattern[0]
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [22]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')
    if not rest: return (seg_pat, saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

In [23]:
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [24]:
def chinese(check_str):#判断字符中是否有汉字存在
   
    for i in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':#这里是从网上搜索得到的，不是很明白为什么！！！！！！！
            return True
    return False


In [25]:
def get_pat(pat):#！！！！！！！！！！！！！！！
    if chinese(pat):
        stored_list = list(jieba.cut(pat))#使用jieba分词将文本切割为列表
        finialout = []
        cut = []
        for i in stored_list:
            if i == '?' and temp == []:
                cut.append(i)
            elif not chinese(i) and temp != []:#占位符后面的x,y
                cut.append(i)
            elif chinese(i) and temp != []:
                finialout.append(''.join(temp))    #对经过jieba分词后的语句，进行再细分，即纯占位符和字母，纯汉字
                finialout.append(i)
                temp = []
            else:
                finialout.append(i)
        if cut != []:finialout.append(''.join(cut))#一个收尾工作，专门处理形如这样的情景：?*x定积分?*y，如果没有这一步，?*y将会被遗弃在cut中
            #最终的输出也将少一部分
        return finialout
    else:
        return pat.split()                            #这个代码或许使用递归会更好，可以尝试用递归重新改写一下

In [26]:
def roughly_match(pat,saying):
    
    for i in pat:
        if chinese(i) and i not in saying:
            return False   #输入与模式不符
    return True

In [27]:
def response2(saying,rule_list):
    for key in rule_list.keys():
        if roughly_match(get_pat(key),get_pat(saying)):
            get_pattern = pat_match_with_seg(get_pat(key),get_pat(saying))
            
            if len(get_pattern[0][1]) != len(get_pat(saying)):
                select = random.choice(rule_list[key])
                decision = subsitite(get_pat(select), pat_to_dict(get_pattern))
                
                if chinese(decision):#由于英文与汉字的拼接方式不同，所以需要判断回答是否含有汉字。这里是最后的收尾工作了
                    decision = ''.join(decision)
                    decision = decision.replace(' ','')
                    return decision
                else:
                    return ' '.join(decision)
        else:
            continue
    print('真有趣，该回家了')#没有相对应的情景模板


#！！！！！！！！！！！！！！需要注意的是，这次的代码中，该句循环：for key in rule_list.keys()其实只能有效的执行一次。在整个循环过程中将会
#带来很多不必要的时间的浪费，或许可以直接进行改写。但是鉴于无法直接从输入得到相应的情景模板，这种效率低下的一次次试错好像又是无法避免的

In [28]:
rule_list = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*x AI? *y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
response2('勃学可能是对的',rule_list)

NameError: name 'ch' is not defined

## 问题四

### 1.这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
优点：可以自动生成一些回复
缺点：这些回复都是基于人为设计好的模板，一旦问题超越这个限定，那么就无法回答。
对于改进的具体方法，现在的我还无能为力，没什么太好的思路。
### 2.什么是数据驱动？数据驱动在这个程序里如何体现？
就我而言，数据驱动是指由大量的数据得到逻辑，再通过这个逻辑处理新的数据。在这个程序里，虽然是依赖于模板，但是具体的对象还是来源于数据的而不是人为设置的。
### 3.数据驱动与 AI 的关系是什么？
就我的认识，数据驱动是AI中的第一步，也是最为至关重要的一步。没有数据驱动就没有AI。